In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import options_trading as opt
import pricing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Simulate Asset Price with Monte Carlo

In [4]:
S0=100
K=100
T=1
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo(S0, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    S=pd.DataFrame()

    for i in range(M):
        St=pd.Series(index=np.arange(0,N,1)).rename(f"{i+1}")
        St[0]=S0
    
        for j in range (1, N+1):
            epsilon=np.random.normal()
            Stn=St[j-1]*np.exp(nudt+volsdt*epsilon)
            St[j]=Stn

        S=pd.concat([S, St], axis=1)

    return S

In [21]:
monte_carlo(S0, T, r, vol, N, M)

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,97.768375,99.601480,101.200619,98.111650,100.566282,99.329402,99.720696,98.714177,97.698759,100.685197,...,99.531524,98.343637,99.855900,99.544698,101.441600,99.591017,98.190810,95.944904,96.698467,97.856663
2,96.662416,98.745168,99.847459,100.880969,100.026409,101.458658,97.572408,96.996126,97.377983,104.309104,...,100.195219,98.317873,96.260970,97.416198,104.991198,100.533393,96.796618,95.525060,101.217434,99.163967
3,97.075415,98.130584,97.206581,99.873091,99.201466,103.033034,96.414278,99.801754,100.829797,106.663421,...,100.490432,98.436115,97.773705,95.976554,103.661108,97.849241,95.080717,92.626285,102.424782,99.060176
4,93.974248,100.326699,98.995034,100.915525,99.477025,103.020489,101.508326,97.403046,97.294758,109.923230,...,102.391508,95.274630,99.776252,97.224124,106.218132,100.249638,99.173547,95.322871,102.590013,99.857230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,102.124115,72.186882,115.324392,105.489562,106.303751,88.178460,78.833397,94.348529,76.817163,87.738056,...,129.259954,82.021297,113.011152,84.512888,73.440301,62.425646,85.040501,73.452742,98.652934,93.165117
97,104.250632,72.268594,116.673187,106.407286,105.449308,85.464734,80.061708,95.017023,78.405511,86.540891,...,133.793208,81.893359,110.624936,81.533980,71.753879,60.154770,85.744623,71.675640,97.271192,92.099071
98,107.043062,71.750945,116.413698,112.480605,104.428249,86.084760,81.233864,93.464098,75.625295,88.295074,...,134.819533,83.254245,109.864022,81.080742,73.055211,60.736319,82.693894,72.148795,98.522167,90.492620
99,105.664800,72.040714,119.222751,110.770700,102.912362,85.327074,80.725419,89.481015,73.847030,90.533005,...,134.993776,84.660843,108.150486,83.619912,74.648439,60.469882,81.442846,71.632702,100.740550,92.208444


In [78]:
S0=100
K=100
T=2
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo_vectorized(S0, T, r, vol, M):
    N=T*252
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)
    epsilon=np.random.normal(0, 1, (N, M))
    growth_factors=np.exp(nudt + volsdt * epsilon)
    prices=S0*pd.DataFrame(growth_factors).cumprod(axis=0)
    S=pd.concat([pd.DataFrame([S0] * M).T, prices], ignore_index=True)
    S.columns=[f"{i+1}" for i in range(M)]

    return S

In [270]:
def monte_carlo_vectorized(S0, T, r, vol, M, antithetic=False):
    """
    S0 : spot price at time t=0
    T : maturity (in years)
    r : risk-free rate
    vol : implied volatility
    M : number of simulations
    antithetic : Boolean type (True to improve precision of results)

    Return a matrix with M columns for each simulation where a simulation represents a path for the asset price
    """

    N=int(round(T*252))   ## 252 times for each trading day in a year
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)

    if antithetic:
        M=int(round(M/2))

    epsilon=np.random.normal(0, 1, (N, M))
    growth_factors=np.exp(nudt + volsdt * epsilon)
    prices=S0*pd.DataFrame(growth_factors).cumprod(axis=0)

    if antithetic:
        growth_factors_ant=np.exp(nudt - volsdt * epsilon)
        prices_ant=S0*pd.DataFrame(growth_factors_ant).cumprod(axis=0)
        prices=pd.concat([prices, prices_ant], axis=1, ignore_index=True)

    S=pd.concat([pd.DataFrame([S0] * prices.shape[1]).T, prices], ignore_index=True)
    S.columns=[f"{i+1}" for i in range(prices.shape[1])]

    return S

In [106]:
S=monte_carlo_vectorized(S0, T, r, vol, M, antithetic=False)
S

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,100.507359,99.929505,100.165528,99.509473,100.568576,98.819091,99.158722,99.071193,99.388435,102.803827,...,99.281387,99.938091,100.183842,99.727129,97.063651,97.992468,99.925373,101.668056,98.938951,99.675189
2,99.441679,100.111937,99.597242,99.119781,99.602105,98.502923,98.523380,100.039969,99.588520,105.040010,...,99.091214,101.997820,99.639249,97.891098,97.888568,97.221902,99.283425,101.552874,100.360310,101.627886
3,98.856265,99.935610,101.747854,99.587791,99.943536,98.213176,99.251721,99.590930,99.292489,106.889531,...,99.724837,103.912188,100.361133,97.419283,96.764294,97.657343,100.204358,101.924993,99.489545,100.854782
4,100.759013,99.183720,102.138821,101.296970,98.515681,95.598377,100.133824,99.940272,99.366367,108.082633,...,100.845386,102.909046,101.343409,96.114018,99.083070,97.688929,100.215302,102.639579,99.703153,100.456051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,82.465395,77.910344,112.891549,67.448847,91.548065,108.351115,100.327511,93.977955,102.853205,77.326700,...,145.289516,123.803653,114.115984,84.058716,95.944089,67.088801,102.769388,94.550370,111.114721,93.775496
501,83.168861,78.015326,113.771116,67.713326,90.125198,106.784549,102.269512,94.886290,103.443699,78.101131,...,147.492368,120.522021,114.596204,81.503197,95.681022,67.780715,102.443327,94.307341,111.036573,94.081209
502,82.202172,77.052823,115.464792,68.449280,89.932726,110.254922,100.961463,95.190071,104.364823,79.574732,...,149.603761,120.418966,117.008600,81.151988,96.650811,68.476648,100.672573,94.983809,110.851418,94.538250
503,82.774509,76.321564,115.466824,67.164028,89.397046,111.818377,100.575761,96.135964,104.368323,79.120084,...,152.861935,118.682061,116.762482,80.842140,97.015155,69.510342,100.717784,97.740971,110.981232,95.963221


### Price Vanilla Option with Monte Carlo

In [6]:
S0=100
K=100
T=1
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo_price(S0, K, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
        CT=max(0,St-K)
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return (C0, SE)

In [81]:
def monte_carlo_price_vectorized(S0, K, T, r, vol, M, opt_type='C'):
    """
    opt_type : "C" for a call and "P" for a put
    """
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, M)
    final_prices=asset_prices.iloc[-1, :]
    
    if opt_type=='C':
        payoffs=np.maximum(final_prices - K, 0)
    elif opt_type=='P':
        payoffs=np.maximum(K-final_prices, 0)
    else:
        print("opt_type should be either 'C' for a call option or 'P' for a put option")

    option_price=np.exp(-r*T)*payoffs.mean()

    return option_price

In [94]:
monte_carlo_price_vectorized(S0=100, K=110, T=1, r=0.02, vol=0.2, M=100000, opt_type='C')

4.925527606922403

In [88]:
opt.black_scholes(r=0.02, S=100, K=110, T=1, sigma=0.2, type="c")

4.94386695723049

### Price a digit option

In [10]:
def digit_option_price(S0, barrier, T, r, vol, digit, N, M):
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]

    payoffs=np.where(final_prices>barrier, digit, 0)

    digit_price=np.exp(-r*T)*payoffs.mean()

    return digit_price

In [11]:
digit_option_price(S0=100, barrier=100, T=1, r=0.02, vol=0.2, digit=10, N=100, M=100000)

4.902659704278397

### Price a Barrier Option

In [ ]:
def barrier_option_price(S0, barrier, K, T, r, vol, N, M, opt_type="C", barrier_type="knock_in"):
    """
    opt_type : "C" for Call and "P" for Put
    barrier_type : "knock_in" or "knock_out"
    """
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]

    trigger=np.where(asset_prices>barrier, 1, 0)
    trigger = np.maximum.accumulate(trigger, axis=0)

    if barrier_type=="knock_in":
        if opt_type=="C":
            payoffs=trigger[-1, :]*np.maximum(final_prices-K, 0)
        elif opt_type=="P":
            payoffs=trigger[-1, :]*np.maximum(K-final_prices, 0)
        else:
            print("opt_type shloud be either 'C' for a call or 'P' for a put")
    elif barrier_type=="knock_out":
        if opt_type=="C":
            payoffs=(1-trigger[-1, :])*np.maximum(final_prices-K, 0)
        elif opt_type=="P":
            payoffs=(1-trigger[-1, :])*np.maximum(K-final_prices, 0)
        else:
            print("opt_type shloud be either 'C' for a call or 'P' for a put")
    else:
        print("barrier_type should be either 'knock_in' or 'knock_out'")

    option_price=np.exp(-r*T)*payoffs.mean()

    return option_price

In [24]:
barrier_option_price(S0=100, barrier=110, K=115, T=1, r=0.02, vol=0.2, N=100, M=100000)

3.583705193408904

In [29]:
def barrier_option_price_v2(S0, barrier, K, T, r, vol, N, M, opt_type="C", barrier_type="knock_in"):
    """
    opt_type : "C" for Call and "P" for Put
    barrier_type : "knock_in" or "knock_out"
    """
    asset_prices = monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices = asset_prices.iloc[-1, :]

    # Return a Series a Boolean : True if barrier was crossed and False if not
    crossed_barrier = np.any(asset_prices>barrier, axis=0)

    # Compute the right crossed_barrier Series according to the barrier_type
    if barrier_type == "knock_in":
        active_payoff = crossed_barrier
    elif barrier_type == "knock_out":
        active_payoff = ~crossed_barrier
    else:
        raise ValueError("barrier_type should be either 'knock_in' or 'knock_out'")

    # Compute the payoff depending on if it is a Call or a Put
    if opt_type == "C":
        payoffs = active_payoff * np.maximum(final_prices - K, 0)
    elif opt_type == "P":
        payoffs = active_payoff * np.maximum(K - final_prices, 0)
    else:
        raise ValueError("opt_type should be either 'C' for a call or 'P' for a put")

    # Compute the price actualized
    option_price = np.exp(-r * T) * payoffs.mean()

    return option_price

In [30]:
barrier_option_price_v2(S0=100, barrier=110, K=115, T=1, r=0.02, vol=0.2, N=100, M=100000)

3.5855722161561383

### Ladder option

In [43]:
def ladder_option_price(S0, strikes, barriers, T, r, vol, N, M, rebates):
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]

    payoff = pd.Series(0, index=asset_prices.columns) 

    for i in range(len(barriers)):
        crossed_barrier=np.any(asset_prices>barriers[i], axis=0)

        payoff+=crossed_barrier*np.maximum(barriers[i]-strikes[i], 0)*rebates[i]
    
    payoff+=np.maximum(final_prices-strikes[-1], 0)
    
    option_price=np.exp(-r*T)*payoff.mean()

    return option_price

In [45]:
ladder_option_price(100, [100, 105, 110], [105, 110], 1, 0.04, 0.1, 100, 100000, [0.05, 0.05])

2.155976899044432

### Price a Lookback Option

In [ ]:
def lookback_option_price(S0, K, T, r, vol, N, M, opt_type='C', strike_type='fixed'):
    """
    opt_type : "C" for a Call option and "P" for a Put option
    strike_type : "fixed" for "float"
    """
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]

    max_prices=asset_prices.max(axis=0)
    min_prices=asset_prices.min(axis=0)

    if strike_type == 'fixed':
        if opt_type == 'C':
            payoff=np.maximum(max_prices-K, 0)
        elif opt_type == 'P':
            payoff=np.maximum(K-min_prices, 0)
        else:
            raise ValueError("opt_type should be either 'C' for a call or 'P' for a put")
    
    elif strike_type == 'float':
        if opt_type == 'C':
            payoff=np.maximum(final_prices-min_prices, 0)
        elif opt_type == 'P':
            payoff=np.maximum(max_prices-final_prices, 0)
        else:
            raise ValueError("opt_type should be either 'C' for a call or 'P' for a put")
        
    else:
        raise ValueError("strike_type should be either 'fixed' or 'float' ")
    
    lookback_option_price=np.exp(-r*T)*payoff.mean()

    return lookback_option_price

In [54]:
lookback_option_price(100, 130, 1, 0.04, 0.2, 100, 100000, opt_type='C')

2.550432802742786

In [128]:
T=1
N=T*252
n=8    # Number of observations

N//n

31

In [115]:
31*8

248

In [ ]:
(N%n)/n  # Ce qu'il faut ajouter à chaque observation pour arriver sur N à la fin des observations, on garde uniquement la partie après la virguel et on regarde si elle est égale à 0, si oui -> nombre entier et donc on ajoute 1 à l'indexation des observations

0.5

In [229]:
asset_prices=monte_carlo_vectorized(S0, T2, r, vol, M, antithetic=True)[round(T1*252):]
asset_prices

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
252,98.530359,107.798128,84.905264,83.644188,102.121354,119.006506,107.389612,78.539560,87.216228,106.781158,...,92.533310,124.617159,154.348453,86.572980,99.840036,103.111676,93.513662,100.825512,104.301785,125.878465
253,98.732876,106.127428,85.343984,84.147176,103.939962,120.308790,105.121994,79.343554,86.833149,104.834764,...,93.269827,127.957247,155.670077,85.398994,101.680135,103.434359,94.785998,100.288122,105.155980,127.412824
254,98.633611,103.354777,86.957094,85.600507,103.935721,120.037208,104.383537,79.024421,88.102192,104.147867,...,93.515165,126.719766,158.246423,86.773403,99.585044,103.128871,95.398449,99.776087,103.838934,128.710685
255,99.563033,102.415148,88.174033,86.068544,103.554092,122.075556,103.586329,78.048799,86.641163,102.765128,...,91.577679,126.025733,156.891682,86.971280,98.499964,103.198648,93.888929,99.325760,103.703921,127.663433
256,98.108504,101.692390,88.752960,85.208305,103.812262,122.946423,103.199914,78.767837,87.410220,102.509398,...,90.384121,127.162590,156.455572,86.431571,99.803715,101.996373,93.450419,98.562783,103.335630,130.186222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,133.043122,96.367825,88.037806,76.774141,125.561235,118.897471,68.624778,69.469160,100.320808,112.911091,...,108.872595,111.334604,138.870066,99.751794,100.499177,120.544606,107.319822,141.229659,85.910741,124.969966
501,130.882092,95.492745,86.825052,77.203117,124.603097,120.293314,68.927259,70.023488,99.829314,111.427800,...,108.133217,110.439004,140.211154,99.144747,99.034272,116.885729,107.800650,139.455744,85.201150,123.720763
502,132.663103,96.977990,86.642877,76.289845,127.300350,120.382668,69.244146,70.220620,100.793412,111.588067,...,106.628064,108.288197,140.521128,99.094775,98.535913,116.376276,106.673038,138.385327,85.569370,123.173822
503,132.718648,97.549024,86.145867,75.866495,125.352515,119.129576,69.708834,71.557253,101.560671,112.716351,...,108.145379,109.508399,140.070535,99.403907,97.166631,114.439813,107.241684,133.469994,85.235141,127.588789


In [ ]:
T1=1
T2=2
n=8

asset_prices=monte_carlo_vectorized(S0, T2, r, vol, M, antithetic=True)[round(T1*252):]

observations=[int(x) for x in list(np.linspace(0, (T2-T1)*252, n+1))][1:]

At_sum=pd.DataFrame()
for obs in observations:
    At_sum=pd.concat([At_sum, asset_prices.iloc[obs]], axis=1)
At_sum=At_sum.sum(axis=1)
At_sum/n

[31, 63, 94, 126, 157, 189, 220, 252]


1       122.679438
2        95.635310
3        69.806014
4       113.196236
5        99.925745
           ...    
996     103.831050
997      86.420926
998     130.431528
999      74.849748
1000    115.115446
Length: 1000, dtype: float64

In [257]:
def asian_option_price(S0, K, T1, T2, n, r, vol, M, antithetic=False, type='arithmetic'):
    """
    type : 'arithmetic' or 'geometric'
    T2-T1 : correspond to our period of observation
    n : nnumber of observations during the period T2-T1
    """
    asset_prices=monte_carlo_vectorized(S0, T2, r, vol, M, antithetic)
    asset_prices=asset_prices[round(T1*252):]

    observations=[int(x) for x in list(np.linspace(0, (T2-T1)*252, n+1))][1:]

    Sum=pd.DataFrame()

    # On parcourt les observations
    for obs in observations:
        Sum=pd.concat([Sum, asset_prices.iloc[obs]], axis=1)

    if type=='arithmetic':
        Sum=Sum.sum(axis=1)/n

    elif type=='geometric':
        Sum=Sum.prod(axis=1)**(1/n)
    
    else:
        raise ValueError("type should be either 'arithmetic' or 'geometric'")

    payoff=np.exp(-r*T2)*np.maximum(Sum-K, 0).mean()

    return payoff

In [255]:
def asian_option_price_v2(S0, K, T1, T2, n, r, vol, M, antithetic=False, type='arithmetic'):
    """
    type : 'arithmetic' or 'geometric'
    T2-T1 : correspond to our period of observation
    n : nnumber of observations during the period T2-T1
    """
    asset_prices=monte_carlo_vectorized(S0, T2, r, vol, M, antithetic)
    asset_prices=asset_prices[round(T1*252):]

    observations=[int(x) for x in list(np.linspace(0, len(asset_prices)-1, n+1))][1:]

    observed_prices=asset_prices.iloc[observations, :]

    if type=='arithmetic':
        average_price=observed_prices.sum(axis=0)/n

    elif type=='geometric':
        average_price=observed_prices.prod(axis=0)**(1/n)
    
    else:
        raise ValueError("type should be either 'arithmetic' or 'geometric'")

    payoff=np.exp(-r*T2)*np.maximum(average_price-K, 0).mean()

    return payoff

In [265]:
asian_option_price_v2(S0=100, K=100, T1=1, T2=5, n=25, r=0.02, vol=0.2, M=100000, antithetic=True, type='geometric')

13.846080803574726

In [266]:
asian_option_price(S0=100, K=100, T1=1, T2=5, n=25, r=0.02, vol=0.2, M=100000, antithetic=False, type='geometric')

13.82778970832866

In [246]:
asian_option_price(S0=100, K=100, T1=1, T2=5, n=25, r=0.02, vol=0.2, M=100000, antithetic=True, type='geometric')

15.400489800117047

In [314]:
def digital_strip(S0, barriers, maturities, r, vol, digit, memory, M, antithetic=False):
    asset_prices=monte_carlo_vectorized(S0, max(maturities), r, vol, M, antithetic)

    num_steps=[int(round(maturity*252)) for maturity in maturities]

    # Extract the final prices for each digit option
    final_prices = asset_prices.iloc[num_steps, :].reset_index(drop=True)

    # Create DataFrames for barriers and times for vectorized operations
    barriers_df = pd.DataFrame(np.tile(barriers, (final_prices.shape[1], 1)).T, index=final_prices.index, columns=final_prices.columns)
    maturities_df = pd.DataFrame(np.tile(maturities, (final_prices.shape[1], 1)).T, index=final_prices.index, columns=final_prices.columns)

    # Initialize matrices for calculations

    memo_df = pd.DataFrame(np.zeros((len(maturities), final_prices.shape[1])), index=final_prices.index, columns=final_prices.columns)
    PV_digits_df = pd.DataFrame(np.zeros((len(maturities), final_prices.shape[1])), index=final_prices.index, columns=final_prices.columns)

    # Calculate the payoff for each simulation path
    PV_digits_df = np.where(final_prices >= barriers_df, 
                            (1 + memo_df) * np.exp(-r * maturities_df) * digit, 0)
    
    # Update memory matrix
    memo_df = np.where(final_prices >= barriers_df, 
                       0, memory * (memo_df + 1))

    PV_digits_df = pd.DataFrame(PV_digits_df, index=final_prices.index, columns=final_prices.columns)
    digitals_strip_price = PV_digits_df.sum(axis=0).mean()

    return digitals_strip_price

In [315]:
digital_strip(S0=100, barriers=[100, 105, 110, 115], maturities=[1, 2, 3, 4], r=0.02, vol=0.2, digit=10, memory=0, M=1000, antithetic=True)

15.910363245283543

In [ ]:
def autocallable (A, S0, H1, H2, H3, c, coupon_period, T, r, vol, M, antithetic=False, conditionnal=False):
    """
    c : value of the coupon
    coupon_period : period for paying the coupon (in months)
    conditionnal : Boolean type
    """
    # on détermine le nombre d'observations
    n=round(T/(coupon_period/12))   # on convertit en années puis on obtient n nombre de coupons

    asset_prices=monte_carlo_price_vectorized(S0, T, r, vol, M, antithetic)
    final_prices=asset_prices.iloc[-1, :]


    observations=[int(x) for x in list(np.linspace(0, len(asset_prices)-1, n+1))][1:]
    observed_prices=asset_prices.iloc[observations, :]

    coupons=pd.DataFrame(np.where((observed_prices>H1) & (observed_prices<H3), A*c, 0), index=observed_prices.index, columns=observed_prices.columns)

    knock_in_barrier=pd.DataFrame(np.where(asset_prices<H1, True, False), index=asset_prices.index, columns=asset_prices.columns).any(axis=0)
    option_payoff=pd.DataFrame(np.where(knock_in_barrier==True, -np.maximum(H1-final_prices, 0), 0), index=asset_prices.columns).T

    redeem_barrier=pd.DataFrame(np.where(observed_prices>H3, True, False), index=observed_prices.index, columns=observed_prices.columns).any(axis=0)
    redeem=pd.DataFrame(np.where(redeem_barrier==True, A, 0), index=observed_prices.columns).T

    payoff=pd.concat([coupons, option_payoff, redeem], axis=0, ignore_index=True).sum(axis=0)
    price=np.exp(-r*T)*payoff.mean()

    return price

In [440]:
def autocallable_V2 (A, S0, H1, H2, H3, c, coupon_period, T, r, vol, M, antithetic=False, conditionnal=False):
    """
    c : value of the coupon
    coupon_period : period for paying the coupon (in months)
    conditionnal : Boolean type
    """
    # on détermine le nombre d'observations
    n=round(T/(coupon_period/12))   # on convertit en années puis on obtient n nombre de coupons

    asset_prices=pricing.monte_carlo_vectorized(S0, T, r, vol, M, antithetic)
    final_prices=asset_prices.iloc[-1, :]


    observations=[int(x) for x in list(np.linspace(0, len(asset_prices)-1, n+1))][1:]
    observed_prices=asset_prices.iloc[observations, :]

    knock_out_barrier=pd.DataFrame(np.where(observed_prices>H3, True, False), index=observed_prices.index, columns=observed_prices.columns)
    knock_out_barrier_full = pd.DataFrame(False, index=asset_prices.index, columns=asset_prices.columns)
    knock_out_barrier_full.iloc[knock_out_barrier.index] = knock_out_barrier
    knock_out_barrier_full=knock_out_barrier_full.cumsum(axis=0).astype(bool)
    knock_out_barrier=knock_out_barrier_full.iloc[observed_prices.index, :]

    first_knock_out_indices = knock_out_barrier.idxmax(axis=0)
    redeem_payoff=pd.DataFrame(np.where(first_knock_out_indices!=0, np.exp(-r*first_knock_out_indices*(1/252))*A, np.exp(-r*T)*A), index=knock_out_barrier.columns).T

    if conditionnal:
        coupons=pd.DataFrame(np.where((observed_prices>H2) & (knock_out_barrier==False), A*c, 0), index=observed_prices.index, columns=observed_prices.columns)
    else:
        coupons=pd.DataFrame(np.where(knock_out_barrier==False, A*c, 0), index=observed_prices.index, columns=observed_prices.columns)
    
    discount_factors = np.exp(-r * (observed_prices.index / 252))
    coupons = coupons.multiply(discount_factors, axis=0)

    knock_in_barrier=pd.DataFrame(np.where((asset_prices<H1)&(knock_out_barrier_full==False), True, False), index=asset_prices.index, columns=asset_prices.columns)
    knock_in_barrier=knock_in_barrier.cumsum(axis=0).astype(bool)
    knock_in_barrier=pd.DataFrame(np.where(asset_prices<H1, True, False), index=asset_prices.index, columns=asset_prices.columns).any(axis=0)
    option_payoff=pd.DataFrame(np.where(knock_in_barrier==True, -np.exp(-r*T)*np.maximum(S0-final_prices, 0), 0), index=asset_prices.columns).T

    payoff=pd.concat([coupons, option_payoff, redeem_payoff], axis=0, ignore_index=True).sum(axis=0)
    price=payoff.mean()

    return price

In [451]:
autocallable_V2 (A=100, S0=100, H1=80, H2=90, H3=110, c=0.04, coupon_period=6, T=2, r=0.02, vol=0.2, M=10000, antithetic=True, conditionnal=False)

99.61291763734368